**сеть Хемминга**

нечёткий поиск по словарю

Евгений Борисов esborisov@sevsu.ru

А.Арустамов, А.Стариков    Ассоциативная память.Применение сетей Хемминга для нечеткого поиска.   
https://basegroup.ru/community/articles/assoc

##  загружаем данные 

In [ ]:
import gzip

def read_list_txt(fname):
    with gzip.open(fname,'rt') as f: 
        return [ w.strip() for w in f.read().split('\n') if w.strip() ]
        
ideal = read_list_txt('data/ideal_u.txt.gz')

max_word_len = max([len(w) for w in ideal]) # максимальная длинна слова словаре
voc_len = len(ideal) # размер словаря

voc_len, max_word_len

In [ ]:
ideal[:7]

## кодируем данные

In [ ]:
import numpy as np
from numpy import random as rng

In [ ]:
# Кодирование строится таким образом, 
# что бы стоящие рядом на компьютерной клавиатуре символы 
# имели близкие по Хеммингу коды.
# Таким образом должно достигаться наиболее эффективное исправление опечаток.
CODE_A = {
   'к':(0,0,0,0,0,),
   'ж':(0,0,0,0,1,),
   'г':(0,0,0,1,0,),
   'щ':(0,0,0,1,1,),
   'ч':(0,0,1,0,0,),
   'х':(0,0,1,0,1,),
   'ю':(0,0,1,1,0,),
   'с':(0,0,1,1,1,),
   'у':(0,1,0,0,0,),
   'б':(0,1,0,0,1,),
   'л':(0,1,0,1,0,),
   'е':(0,1,0,1,1,),
   'о':(0,1,1,0,0,),
   'р':(0,1,1,0,1,),
   'н':(0,1,1,1,0,),
   'й':(0,1,1,1,1,),
   'т':(1,0,0,0,0,),
   'з':(1,0,0,0,1,),
   'ы':(1,0,0,1,0,),
   'п':(1,0,0,1,1,),
   'ф':(1,0,1,0,0,),
   'ш':(1,0,1,0,1,),
   'м':(1,0,1,1,0,),
   'э':(1,0,1,1,1,),
   'ъ':(1,1,0,0,0,),
   'д':(1,1,0,0,1,),
   'в':(1,1,0,1,0,),
   'а':(1,1,0,1,1,),
   'ц':(1,1,1,0,0,),
   'и':(1,1,1,0,1,),
   'я':(1,1,1,1,0,),
   'ь':(1,1,1,1,1,)
}

CODE_V = { CODE_A[a]:a for a in CODE_A }

def encode(w): return  [ CODE_A[a] for a in list(w.lower()) ]
def decode(c): return  [ CODE_V[v] for v in c ]

In [ ]:
code_len = len(CODE_A['а']) # длинна кода символа

In [ ]:
# заменяем буквы на коды
def encode_text(text):
    return [ encode(w) for w in text]

# обрезаем длиные слова
def strip_text(text,max_word=0): 
    return [ w[:max_word_len] for w in text ] 

# дополняем коды коротких слов нулями
def pad_code(x,max_word=0): 
    code_len = len(x[0][0]) # длинна кода
    # максимальная длинна слова
    mwl = max([len(v) for v in x]) if max_word<1 else max_word
    # дополнение нулями
    z = [(0,)*code_len]*mwl
    # дополняем короткое слово
    return [ v + z[:(mwl-len(v))] for v in x ]

# собираем датасет из списка слов text,
# max_word - ограничение максимальной длины слова (0 - нет ограничений)
def make_dataset(text,max_word=0):
    t = strip_text(text,max_word) 
    x = encode_text(t)
    x = pad_code(x,max_word)
    return np.array([ sum(v,()) for v in x ])
    

In [ ]:
x_train = make_dataset(ideal)   

# масштабируем в [-1,+1]
x_train = x_train*2-1

display(x_train.shape)

In [ ]:
display(x_train)

## загружаем память сети

сеть Хэминга

![neural-net-hamming](http://mechanoid.su/content/neural-net-hamming-classifier.html/nnet.png)

In [ ]:
class HammingNet:
    
    def __init__(self):
        self._weight_lin = 0.
        self._weight_hop = 0.
        
    def fit(self,x):
        self._weight_lin =  0.5*x.T
        
        n_samples,_ = x.shape # количество учебных примеров 
        c = 1./(2.*n_samples) # коэффициент торможения
        # веса для нейронов второго слоя
        self._weight_hop = -c*(np.ones(n_samples)-np.eye(n_samples)) + np.eye(n_samples)
        
        return self
    
    def forward(self,x,max_iter=8):
        o = x.dot(self._weight_lin)
        for n in range(max_iter):
            o_ = o.copy() # сохраняем состояние
            o = self._forward_step(o) # переходим в новое состояние
            # если состояние не изменилось то завершаем
            if np.all(o==o_): break
        return n,o        
    
    def _forward_step(self,x):
        return np.max( [x.dot(self._weight_hop), np.zeros(x.shape)],axis=0 )
    
    def __call__(self,x):
        return self.forward(x)
    
    def predict(self,x):
        n,o = self.forward(x)
        return n,np.argmax(o,axis=1)

In [ ]:
model = HammingNet().fit(x_train)

## тестируем

In [ ]:
test = read_list_txt('data/test_u.txt.gz')
display( len(test) )
display( test[:7] )

In [ ]:
max_word_len = max([len(w) for w in ideal])
x_test = make_dataset(test,max_word=max_word_len)    
x_test = x_test*2-1
display( x_test.shape )

In [ ]:
from tabulate import tabulate 

_,p = model.predict(x_test)
res = [ [t, ideal[p[i]] ] for i,t in enumerate(test) ]

tabulate(res, tablefmt='html')